### Import bibliotek 

In [1]:
import os
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException, StaleElementReferenceException
import time

### Definicja narzędzi pomocniczych

In [2]:
def extract(ancestor, selector=None, attribute=None, many=False):
    if selector:
        if many:
            if attribute:
                return [item[attribute].strip() for item in ancestor.select(selector)]
            return [item.text.strip() for item in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        try:
            return ancestor[attribute]
        except TypeError:
            return None
    return ancestor.text.strip()

In [42]:
opinion_schema = {
    "stars": ("span.a-icon-alt",),
    "content": ("span.review-text-content",)
}

### Logowanie do Amazona

In [43]:
product_id = input("Podaj kod produktu: ") #B08PNMFT47

In [44]:
with open("./credentials.json", "r") as jf:
     credentials = json.load(jf)
ACCOUNT = credentials["ACCOUNT"]
PASSWORD = credentials["PASSWORD"]

In [45]:
next_page = f"https://www.amazon.com/product-reviews/{product_id}"
path_to_chromedriver = r"C:\Users\jedre\Downloads\chromedriver-win64\chromedriver.exe"
s = Service(path_to_chromedriver)

driver = webdriver.Chrome(service=s)
driver.get(next_page)
driver.maximize_window()

try:
    driver.find_element(by='xpath', value='/html/body/div/div[1]/div[3]/div/div/form/div/div/span/span/button').click()
except (NoSuchElementException):
    pass
driver.refresh()
WebDriverWait(driver, 20).until(EC.element_to_be_clickable(driver.find_element(by='id', value='ap_email_login'))).send_keys(ACCOUNT)
driver.find_element(by='xpath', value='//*[@id="continue"]/span/input').click()
WebDriverWait(driver, 20).until(EC.element_to_be_clickable(driver.find_element(by='name', value='password'))).send_keys(PASSWORD)
driver.find_element(by='xpath', value='//*[@id="signInSubmit"]').click()


### Pobranie z Amazon.com opinii o wskazanym produkcie

In [30]:
all_opinions = []
while True:
    opinions = driver.find_elements(by=By.CSS_SELECTOR, value="li.review.aok-relative")
    print(len(opinions))
    for opinion in opinions:
        try:
            opinion_html = opinion.get_attribute('innerHTML')
            opinion_dom = BeautifulSoup(opinion_html, 'html.parser')
            single_opinion = {
                key: extract(opinion_dom, *value)
                for key, value in opinion_schema.items()
            }
            all_opinions.append(single_opinion)
        except StaleElementReferenceException:
            print("StaleElementReferenceException")
            continue
    try:
        time.sleep(1)
        driver.find_element(by=By.CSS_SELECTOR, value='li.a-last a').click()
    except NoSuchElementException:
        next_page = None
        break

10
10


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=143.0.7499.193); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#staleelementreferenceexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff62f2588d5
	0x7ff62f258930
	0x7ff62f03165d
	0x7ff62f0392cc
	0x7ff62f03c3b4
	0x7ff62f03c47f
	0x7ff62f0918ca
	0x7ff62f08f40f
	0x7ff62f08c417
	0x7ff62f08b338
	0x7ff62f07ce7f
	0x7ff62f0b1fda
	0x7ff62f07c746
	0x7ff62f0dac97
	0x7ff62f07ac29
	0x7ff62f07ba93
	0x7ff62f570620
	0x7ff62f56af60
	0x7ff62f5896c6
	0x7ff62f275dd4
	0x7ff62f27ed7c
	0x7ff62f261ff4
	0x7ff62f2621a5
	0x7ff62f247ed2
	0x7ffd2c98e8d7
	0x7ffd2d9cc53c


In [31]:
driver.get(next_page)
driver.find_element(by="xpath", value='//*[@id="cm_cr-product_info"]/div/div[2]/div/div/div[2]/div[1]/h1/a').click()
page_html = driver.page_source
page_dom = BeautifulSoup(page_html, 'html.parser')
brand = extract(page_dom, "tr.po-brand > td:nth-child(2)")
model = extract(page_dom, "tr.po-model_name > td:nth-child(2)")

In [32]:
print(f"Zbieranie opinii dla produktu: {brand} {model}")

Zbieranie opinii dla produktu: Apple iPhone 12 Pro


### Zapisanie pobranych opinii do plików tekstowych

In [36]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")
if not os.path.exists("./opinions/en"):
    os.mkdir("./opinions/en")

In [35]:
for idx, single_opinion in enumerate(all_opinions):
    rating = int(round(float(single_opinion["stars"].split(" ")[0])/5,3)*100)
    sentiment = "P" if rating >= 50 else "N"
    filename = f"./opinions/en/{brand.replace(' ', '')}_{model.replace(' ', '')}_{rating:03}_{sentiment}_{idx:03}.txt"
    with open(filename, "w", encoding="utf-8") as tf:
        tf.write(single_opinion["content"])